# Functions

## obj_prepare_path

In [1]:
def obj_prepare_path( obj, salt, status )
  def helper_parse_path( str )
    if str[ str.length - 1, 1 ] != '/'
      str = str + '/'
    end
    return str
  end

  def draw_obj_line_edge( l, offset )
      str = ''
      for i in 1..( ( l - 1 ) * offset )
        str += ' '
      end
      if l > 1
        str += "┗"
        str += "━"
        str += " "
      else
        str += '    '
      end
      return str
  end
  
  def draw_obj_path_local( str, l, offset )
    result = ''
    for i in 0..( ( l - 1 ) * offset )
      result += ' '
    end
    result += ''
    result += str
    return result
  end

  def helper_obj_path( name, salt, k, f=nil )
    str = ''
    str += draw_obj_line_edge( name[ 1, name.length ].to_i, 4 )
    str += 'hash[:path]'
    if k.length == 0

    else
      for i in 0..k.length-1
        str += '[:children]'
        str += '[:'
        str += k[ i ].to_s
        str += ']'      
      end
    end

    if !f.nil?
      str += '[:files][:'
      str += f.to_s
      str += '][:full]'    
    else
      str += '[:full]'
    end
    return str
  end
  
  def helper_insert_salt( salt, str )
    if salt[ 0, 1 ] == ''
      
    else
      if salt[ 0, 1 ] == '-'
      else
        salt = '-' + salt      
      end
    end

    str = str.gsub( "{{SALT}}", salt )
    return str
  end

  def helper_parse_path( str )
    if str[ str.length - 1, 1 ] != '/'
      str = str + '/'
    end
    return str
  end

  def draw_obj_line_edge( l, offset )
      str = ''
      for i in 1..( ( l - 1 ) * offset )
        str += ' '
      end
      if l > 1
        str += "┗"
        str += "━"
        str += " "
      else
        str += '    '
      end
      return str
  end
  
  def draw_obj_path_local( str, l, offset )
    result = ''
    for i in 0..( ( l - 1 ) * offset )
      result += ' '
    end
    result = result[ 1, result.length ]
    
    if str.index('.') != nil
      result += 'File: '
    else
      result += 'Folder: '
    end
    result += str
    return result
  end

  def helper_obj_path( name, salt, k, f=nil )
    str = ''
    str += draw_obj_line_edge( name[ 1, name.length ].to_i, 4 )
    str += ''
    str += 'hash[:path]'
    if k.length == 0

    else
      for i in 0..k.length-1
        str += '[:children]'
        str += '[:'
        str += k[ i ].to_s
        str += ']'      
      end
    end

    if !f.nil?
      str += '[:files][:'
      str += f.to_s
      str += '][:full]'    
    else
      str += '[:full]'
    end
    return str
  end
  
  def helper_insert_salt( salt, str )
    if salt[ 0, 1 ] == ''
      
    else
      if salt[ 0, 1 ] == '-'
      else
        salt = '-' + salt      
      end
    end

    str = str.gsub( "{{SALT}}", salt )
    return str
  end

  mode = {
    :general => nil,
    :hash => nil,
    :path => nil
  }
    
  case status
    when 0
      mode[:general] = false
      mode[:hash] = false
      mode[:path] = false
    when 1
      mode[:general] = true
      mode[:hash] = true
      mode[:path] = false
    when 2
      mode[:general] = true
      mode[:hash] = false
      mode[:path] = true
    when 3
      mode[:general] = true
      mode[:hash] = true
      mode[:path] = true
  end
  
  
  mode[:general] ? puts( 'TREE OVERVIEW' ) : ''
  
  obj[:full] = ''
  obj[:full] += obj[:root]
  obj[:full] += helper_parse_path( helper_insert_salt( salt, obj[:name] ) )
  
  mode[:hash] ? puts( helper_obj_path( 'l1', salt, [ ] ) ) : ''
  mode[:path] ? puts( draw_obj_path_local( obj[:full], 2, 4 ) ) : ''

  obj[:children].keys.each { | l2 | 
      obj[:children][ l2 ][:full] = ''
      obj[:children][ l2 ][:full] += obj[:full]
      obj[:children][ l2 ][:full] += helper_parse_path( helper_insert_salt( salt, obj[:children][ l2 ][:name] ) )
      mode[:hash] ? puts( helper_obj_path( 'l2', salt, [ l2 ] ) ) : ''
      mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:full], 3, 4 ) ) : ''
      FileUtils.mkdir_p obj[:children][ l2 ][:full]

      if !obj[:children][ l2 ][:files].nil?
        obj[:children][ l2 ][:files].keys.each { | f1 | 
          obj[:children][ l2 ][:files][ f1 ][:full] = '' 
          obj[:children][ l2 ][:files][ f1 ][:full] += obj[:children][ l2 ][:full] 
          obj[:children][ l2 ][:files][ f1 ][:full] += helper_insert_salt( salt, obj[:children][ l2 ][:files][ f1 ][:name] )
          mode[:hash] ? puts( helper_obj_path( 'l2', salt, [ l2 ], f1 ) ) : ''
          mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:files][ f1 ][:full], 3, 4 ) ) : ''
        }
      end

      if !obj[:children][ l2 ][:children].nil?
        obj[:children][ l2 ][:children].keys.each { | l3 | 
          obj[:children][ l2 ][:children][ l3 ][:full] = ''
          obj[:children][ l2 ][:children][ l3 ][:full] += obj[:children][ l2 ][:full]
          obj[:children][ l2 ][:children][ l3 ][:full] += helper_parse_path( helper_insert_salt( salt, obj[:children][ l2 ][:children][ l3 ][:name] ) )

          FileUtils.mkdir_p obj[:children][ l2 ][:children][ l3 ][:full]
          mode[:hash] ? puts( helper_obj_path( 'l3', salt, [ l2, l3 ] ) ) : ''
          mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:children][ l3 ][:full], 4, 4 ) ) : ''

          if !obj[:children][ l2 ][:children][ l3 ][:files].nil?
            obj[:children][ l2 ][:children][ l3 ][:files].keys.each { | f2 | 
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] = ''
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] += obj[:children][ l2 ][:children][ l3 ][:full]
              obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full] += helper_insert_salt( salt, obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:name] )
              mode[:hash] ? puts( helper_obj_path( 'l3', salt, [ l2, l3 ], f2 ) ) : ''
              mode[:path] ? puts( draw_obj_path_local( obj[:children][ l2 ][:children][ l3 ][:files][ f2 ][:full], 4, 4 ) ) : ''
            }
          end
        }
      end
  }
  return obj
end

:obj_prepare_path

## download_items

In [2]:
def download_items( obj, key, suffix, delay )
  def download_item( index, key, obj )
    url = ''
    url += obj[:meta][:larvalabs][ key ]
    url += '?index='
    url += index.to_s

    uri = URI( url )
    response = Net::HTTP.get( uri )
    if key == :code
      doc = Nokogiri::HTML( response ) 
      str = doc.text.strip!
    else
      str = response
    end
    return str
  end
  
  for index in 1..512
    index%10 == 0 ? print('.') : ''
    sleep( delay )
    item = download_item( index, key, obj )

    path = ''
    path += obj[:path][:children][:data][:children][ key ][:full]
    path += index.to_s
    path += '.' + suffix
    
    File.open( path, "w" ) { | f | f.write( item ) } 
  end
  return true
end

:download_items

## ascii_to_svg

In [3]:
def ascii_to_svg()
  def params_prepare( ascii, _length ) 
      params = {
        canvas: {
            size:{
              x: 880,
              y: nil,
            },
            margin: {
                left: 120,
                top: 120,
                right: 120,
                bottom: 120
            }
        },
        grid: {
            x: {
                offset: nil,
                length: nil
            },
            y: {
                offset: nil,
                length: nil
            },
            size: {
                x: nil,
                y: nil
            }
        },
        cell: {
            x: {
              offset: 0
            },
            y: {
              offset: 0
            },
            size: {
              x: nil,
              y: nil
            }
        },
        instructions: {
          "\\": [ "\\" ],
          "/": [ "/" ],
          "X": [ "X", "x" ],
          "-": [ "-" ],
          "|": [ "|", "1" ],
          "O": [ "O", "o", "0" ],
          "+": [ "+" ],
          "#": [ "#" ]
        },
        style: {
          line: {
            stroke: {
              width: 2.0,
              color: 'rgb(0,0,0)',
              opacity: 1.0,
              linecap: 'square' # butt / round / square
            }
          },
          ellipse: {
              stroke: {
                width: 2.0,
                color: 'rgb(0,0,0)',
                opacity: 1.0,
                linecap: 'square'
              },
              fill: 'none'
          },
          rectangle: {
            fill: {
              color: 'rgb(0,0,0)',
              opacity: 1.0
            }
          },
          canvas: {
            fill: {
              color: 'rgb(255,255,255)',
              opacity: 1.0
            }          
          }
        }
    }
    
    params[:grid][:x][:length] = _length
    
    lines = ascii.chars.each_slice( params[:grid][:x][:length] ).map( &:join )
    params[:grid][:y][:length] = lines.length

    params[:grid][:x][:offset] = params[:canvas][:margin][:left] + params[:cell][:x][:offset] / 2
    params[:grid][:y][:offset] = params[:canvas][:margin][:top] + params[:cell][:x][:offset] / 2

    tmp = [ :left, :right ].map{ | k | params[:canvas][:margin][ k ] }.sum
    params[:grid][:size][:x] = params[:canvas][:size][:x] - tmp

    tmp = ( params[:cell][:x][:offset] * params[:grid][:x][:length] )
    params[:cell][:size][:x] = ( params[:grid][:size][:x] - tmp ) / params[:grid][:x][:length]
    
    tmp = [ :top, :bottom ].map{ | k | params[:canvas][:margin][ k ] }.sum
    params[:canvas][:size][:y] = ( params[:cell][:size][:x] * params[:grid][:y][:length] ) + tmp
    params[:grid][:size][:y] = params[:canvas][:size][:y] - tmp
    
    tmp = ( params[:cell][:y][:offset] * params[:grid][:y][:length] )
    params[:cell][:size][:y] = ( params[:grid][:size][:y] - tmp ) / params[:grid][:y][:length]
    
    return [ params, lines ]
  end

  def cell_instructions( char, params )
    selector = nil
    params[:instructions].keys.each do | key |
       params[:instructions][ key ].include?( char ) ? selector = key : ''
    end

    results = []
    case( selector )
      when :"\\"
        results.push( [ 'line', :top__left, :bottom__right ] )
      when :"/"
        results.push( [ 'line', :top__right, :bottom__left ] )
      when :"X"
        results.push( [ 'line', :top__right, :bottom__left ] )
        results.push( [ 'line', :top__left, :bottom__right ] )
      when :"-"
        results.push( [ 'line', :center__left, :center__right ] )
      when :"|"
        results.push( [ 'line', :top__center, :bottom__center ] )
      when :"+"
        results.push( [ 'line', :center__left, :center__right ] )
        results.push( [ 'line', :top__center, :bottom__center ] )
      when :"O"
        results.push( [ 'ellipse', :center__center, nil ] )
      when :"#"
        results.push( [ 'rectangle', :top__left, nil ] )
    end
    return results
  end


  def cell_position( x, y, params )
    pos = {
      top:{
        left: nil,
        center: nil,
        right: nil
      },
      center: {
        left: nil,
        center: nil,
        right: nil,
      },
      bottom: {
        left: nil,
        center: nil,
        right: nil
      }
    }


    x = ( x * ( params[:cell][:size][:x] + params[:cell][:x][:offset] ) ) + params[:grid][:x][:offset]
    y = ( y * ( params[:cell][:size][:y] + params[:cell][:y][:offset] ) ) + params[:grid][:y][:offset]

    half_x = params[:cell][:size][:x] / 2
    half_y = params[:cell][:size][:y] / 2

    pos[:top][:left] = [ x, y ]
    pos[:top][:center] = [ x + half_x, y ]
    pos[:top][:right] = [ x + params[:cell][:size][:x], y ]

    pos[:center][:left] = [ x, y + half_y ]
    pos[:center][:center] = [ x + half_x, y + half_y ]
    pos[:center][:right] = [ x + params[:cell][:size][:x], y + half_y ]

    pos[:bottom][:left] = [ x, y + params[:cell][:size][:y] ]
    pos[:bottom][:center] = [ x + half_y, y + params[:cell][:size][:y] ]
    pos[:bottom][:right] = [ x + params[:cell][:size][:x], y + params[:cell][:size][:y] ]

    return pos
  end


  def cell_svg( instructions, position, params )
    str = ''
    instructions.each do | instruction |
      case instruction[ 0 ]
        when 'line'
          keys = instruction
            .drop( 1 )
            .map { | c | 
                keys = c
                  .to_s.split( '__' )
                  .map { | c | c.to_sym }

            }

          x1, y1 = position[ keys[ 0 ][ 0 ] ][ keys[ 0 ][ 1 ] ]
          x2, y2 = position[ keys[ 1 ][ 0 ] ][ keys[ 1 ][ 1 ] ]

          s_width = params[:style][:line][:stroke][:width].to_s
          s_color = params[:style][:line][:stroke][:color].to_s
          s_opacity = params[:style][:line][:stroke][:opacity].to_s
          s_linecap = params[:style][:line][:stroke][:linecap].to_s

          str += "<line x1=\"#{x1}\" y1=\"#{y1}\" x2=\"#{x2}\" y2=\"#{y2}\" style=\"stroke-width: #{s_width};stroke: #{s_color};stroke-opacity: #{s_opacity};stroke-linecap: #{s_linecap};\" transform=\"matrix(1,0,0,1,0,0)\" />"

        when 'ellipse'
          keys = instruction
            .drop( 1 )
            .map { | c | 
                keys = c
                  .to_s.split( '__' )
                  .map { | c | c }
            }

          cx, cy = position[ keys[ 0 ][ 0 ].to_sym ][ keys[ 0 ][ 1 ].to_sym ]
          rx = params[:cell][:size][:x] / 2
          ry = params[:cell][:size][:y] / 2

          s_width = params[:style][:ellipse][:stroke][:width]
          s_color = params[:style][:ellipse][:stroke][:color]
          s_opacity = params[:style][:ellipse][:stroke][:opacity]
          s_linecap = params[:style][:ellipse][:stroke][:linecap]
          e_fill = params[:style][:ellipse][:fill]
          str += "<ellipse cx=\"#{cx}\" cy=\"#{cy}\" rx=\"#{rx}\" ry=\"#{ry}\" style=\"stroke-width: #{s_width};stroke: #{s_color};stroke-opacity: #{s_opacity};stroke-linecap: #{s_linecap};; fill: #{e_fill}\" transform=\"matrix(1,0,0,1,0,0)\" />"

        when 'rectangle'
          keys = instruction
            .drop( 1 )
            .map { | c | 
                keys = c
                  .to_s.split( '__' )
                  .map { | c | c }
          }

          x, y = position[ keys[ 0 ][ 0 ].to_sym ][ keys[ 0 ][ 1 ].to_sym ]
          r_fill = params[:style][:rectangle][:fill][:color]
          r_opacity = params[:style][:rectangle][:fill][:opacity]

          width = params[:cell][:size][:x]
          height = params[:cell][:size][:y]

          str += "<rect x=\"#{x}\" y=\"#{y}\" width=\"#{width}\" height=\"#{height}\" style=\"fill: #{r_fill}; fill-opacity: #{r_opacity}\" transform=\"matrix(1,0,0,1,0,0)\" />"
      end
    end
    return str
  end


  def generate( lines, params )
    svg = ''
    width = params[:canvas][:size][:x]
    height = params[:canvas][:size][:y]
    fill = params[:style][:canvas][:fill][:color]
    fill_opacity = params[:style][:canvas][:fill][:opacity].to_s

    svg += "<svg xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" xmlns:jfreesvg=\"http://www.jfree.org/jfreesvg/svg\" width=\"#{width}\" height=\"#{height}\" text-rendering=\"auto\" shape-rendering=\"auto\">
  <defs></defs>"
    svg += "<rect x=\"0\" y=\"0\" width=\"#{width}\" height=\"#{height}\" style=\"fill: #{fill}; fill-opacity: #{fill_opacity}\" transform=\"matrix(1,0,0,1,0,0)\" />"
    svg += lines.clone
    svg += "</svg>"
    return svg
  end

  def from_string( ascii, _length )
    params, lines = params_prepare( ascii, _length )
    symbols = ''
    for y in 0..params[:grid][:y][:length] - 1
      line = lines[ y ]
      chars = line.split( '' )
      for x in 0..params[:grid][:x][:length] - 1
        char = line[ x ]
        instructions = cell_instructions( char, params )
        position = cell_position( x, y, params )
        symbols += cell_svg( instructions, position, params )      
      end
    end

    result = generate( symbols, params )
  end
end

:ascii_to_svg

## check_uniqness

In [4]:
def check_uniqness( o, c, obj )
  original = ''
  original += obj[:path][:children][:data][:children][:svg][:full]
  original += o

  copy = ''
  copy += c
  
  files = []
  [ original, copy ].each do | path |
    file = File.read( path )
    file.gsub!("\n", '')
    file.gsub!(' ', '')
    
    files.push( file ) 
  end
  
  return files[ 0 ].eql? files[ 1 ]
end

:check_uniqness

## get_allow_list

In [5]:
def get_allow_list( params )
  available = []
  generated = []

  params.keys.each do | lvl1 |
    if params[ lvl1 ].class.to_s.eql? 'Hash'
      params[ lvl1 ].keys.each do | lvl2 |
        if params[ lvl1 ][ lvl2 ].class.to_s.eql? 'Hash'
          params[ lvl1 ][ lvl2 ].keys.each do | lvl3 |
            if params[ lvl1 ][ lvl2 ][ lvl3 ].class.to_s.eql? 'Hash'
              params[ lvl1 ][ lvl2 ][ lvl3 ].keys.each do | lvl4 |
                if params[ lvl1 ][ lvl2 ][ lvl3 ][ lvl4 ].class.to_s.eql? 'Hash'
                  puts 'HERE'
                else
                  str = ( lvl1.to_s + '__' + lvl2.to_s + '__' + lvl3.to_s + '__' + lvl4.to_s ).to_sym
                  if !params[ lvl1 ][ lvl2 ][ lvl3 ][ lvl4 ].nil?
                    available.push( str )
                  else
                    generated.push( str )
                  end
                end
              end
            else
              str = ( lvl1.to_s + '__' + lvl2.to_s + '__' + lvl3.to_s ).to_sym
              if !params[ lvl1 ][ lvl2 ][ lvl3 ].nil?
                available.push( str )
              else
                generated.push( str )
              end
            end
          end
        else
          str = ( lvl1.to_s + '__' + lvl2.to_s ).to_sym
          if !params[ lvl1 ][ lvl2  ].nil? 
            available.push( str )
          else
            generated.push( str )
          end
        end
      end
    else
      str = lvl1
      if !params[ lvl1 ].nil?
        available.push( str )
      else
        generated.push( str )
      end
    end

  end
  return [ available, generated ]
end

:get_allow_list

## markdown_options_tables

In [7]:
def markdown_options_tables( lists, params )
  def pick_values( pick, params )
    keys = pick
      .to_s
      .split( '__' )
      .map{ | a | a.to_sym }
    result = nil
    case( keys.length)
      when 1
        result = params[ keys[ 0 ] ]
      when 2
        result = params[ keys[ 0 ] ][ keys[ 1 ] ]
      when 3
        result = params[ keys[ 0 ] ][ keys[ 1 ] ][ keys[ 2 ] ]
      when 4
        result = params[ keys[ 0 ] ][ keys[ 1 ] ][ keys[ 2 ] ][ keys[ 3 ] ]
    end

    return result
  end


  sort = {}
  lists[ 0 ].each do | key |
    category = key.to_s.split( '__' )[ 0 ].to_sym
    !sort.key?( category ) ? sort[ category ] = [] : ''
    sort[ category ].push( key )
  end


  tables = {}
  alphabet = 'A'.upto( 'Z' ).to_a
  sort.keys.each.with_index do | category, index |
    char = alphabet[ index ]
    str = ''
    str += "| Nr | Name | Key | Default | Type | Description |\n" 
    str += "| :-- | :-- | :-- | :-- | :-- | :-- |\n"
    sort[ category ].each.with_index do | key, index |
      nr = "#{char}.#{index + 1}."
      name = key.to_s.split('__').drop( 1 ).map { | a | a[ 0, 1 ].upcase + a[ 1, a.length ]}.join( " " )
      default = pick_values( key, params )
      type = default.class.to_s
      type.eql?('String') ? default = "\"#{default}\"" : ''
      default = "`#{default}`"
      str += "| #{nr} | #{name} |:#{key} | #{default} | #{type} | |\n"
    end
    tables[ category ] = str
  end


  str = "## Options\n"
  tables.keys.each.with_index do | key |
    name = key.to_s
    name = name[ 0, 1 ].upcase + name[ 1, name.length ] 
    str += "### " + name + "\n"
    str += tables[ key ]
    str += "\n\n"
  end
  str = str[ 0, str.rindex("\n\n\n")]
  
  return str
end

:markdown_options_tables

## google_fonts_download

In [8]:
def google_fonts_download( font, style, subset, file_name )
  def self.str_difference( a, b )
    a = a.to_s.downcase.split( '_' ).join( '' )
    b = b.to_s.downcase.split( '_' ).join( '' )
    longer = [ a.size, b.size ].max
    same = a
      .each_char
      .zip( b.each_char )
      .select { | a, b | a == b }
      .size
    ( longer - same ) / a.size.to_f
  end
  
  def unzip_file( file, destination, file_name )
    Zip::ZipFile.open( file ) do |zip_file|
     zip_file.each do | f |
       f_path=File.join( destination, file_name )  
       FileUtils.mkdir_p( File.dirname( f_path ) )
       zip_file.extract( f, f_path ) unless File.exist?( f_path )
     end
    end
  end

  
  url = 'https://google-webfonts-helper.herokuapp.com/api/fonts'
  uri = URI( url )
  response = Net::HTTP.get( uri )
  fonts_all = JSON.parse( response )

  fonts_subset = fonts_all.select { | a | a["subsets"].include? subset }

  id = fonts_subset
    .map { | a | { name: a["id"], score: str_difference( font, a["id"] ) } }
    .min_by { | h | h[:score] }

  font = fonts_subset
    .find { | a | a["id"].eql? id[:name] }

  variant = font['variants']
    .map { | a | { name: a, score: str_difference( style.to_s, a.to_s ) } }
    .min_by { | h | h[:score] }

  puts "Font: #{font["id"]} #{variant[:name]} (#{subset})"
  
  url = ''
  url += 'https://google-webfonts-helper.herokuapp.com'
  url += "/api/fonts/#{id[:name]}?download=zip&subsets=latin&formats=ttf&variants=#{variant[:name]}"

  puts url
  uri = URI( url )
  response = Net::HTTP.get( uri )

  path_zip = '/Users/andreasbanholzer/Desktop/a/test.zip'
  
  File.open( path_zip, "w" ) { | f | f.write( response ) }
  unzip_file( path_zip, File.dirname( path_zip ), file_name )
  File.delete( path_zip )
end

:google_fonts_download

# Execute

In [9]:
require 'net/http'
require 'active_support/core_ext/hash/indifferent_access'
require 'uri'
require 'nokogiri'

true

In [70]:
hash = {
  path: {
    root: '.',
    name: '/',
    full: nil,
    children: {
      data: {
        name: '0-data',
        full: nil,
        children: {
          code: {
            name: '0-code',
            full: nil
          },
          svg: {
            name: '1-svg',
            full: nil
          },
          generate: {
            name: '2-generate',
            full: nil
          }
        }
      }
    } 
  },
  meta: {
    larvalabs: { 
      code: 'https://www.larvalabs.com/autoglyphs/rawglyph',
      svg: 'https://www.larvalabs.com/autoglyphs/glyphimage'
    }
  }
}

salt = Time.now.getutc.to_i.to_s
hash[:path] = obj_prepare_path( hash[:path], salt, 1 )
puts

TREE OVERVIEW
    hash[:path][:full]
    ┗━ hash[:path][:children][:data][:full]
        ┗━ hash[:path][:children][:data][:children][:code][:full]
        ┗━ hash[:path][:children][:data][:children][:svg][:full]
        ┗━ hash[:path][:children][:data][:children][:generate][:full]



## Download Code

In [44]:
download_items( hash, 5 )

...................................................

true

## Download SVG

In [28]:
download_items( hash, :svg, '.svg', 5 )

...................................................

true

## Generate svg

In [12]:
index = 512
p = ''
p += hash[:path][:children][:data][:children][:code][:full]
p += '/'
p += index.to_s
p += '.txt'
ascii = File.read( p ).gsub( "\n", "" )
svg = ascii_to_svg.from_string( ascii, 64 )

path = 'test.svg'
File.open( path, "w" ) { | f | f.write( svg ) }
check_uniqness( index.to_s + '.svg', path, hash )

#hash[:path][:children][:data][:children][:generate][:full]

true

## Test Module

In [47]:
require '/Users/andreasbanholzer/PROJEKTE/2021-06-11--nft/1/2-ruby/ascii_to_svg/lib/ascii_to_svg.rb'
test = AsciiToSvg.from_string( ascii, 64 )
puts

## Generate Readme

In [ ]:
require '/Users/andreasbanholzer/PROJEKTE/2021-05-27--github-profile/3/markdown_titles_to_svg/lib/markdown_titles_to_svg.rb'
params = MarkdownTitlesToSvg.get_options
lists = get_allow_list( params )
t = markdown_options_tables( lists, params )
t.split("\n").each { | tt | puts tt}
#puts

## MarkdownHeadlinesToSVG

In [13]:
require '/Users/andreasbanholzer/PROJEKTE/2021-05-27--github-profile/3/markdown_titles_to_svg/lib/markdown_titles_to_svg.rb'
params = MarkdownTitlesToSvg.get_options()
lists = get_allow_list( params )

[[:github__source, :font__text_align, :font__bold, :font__mode__h1, :font__mode__default, :font__google_fonts__h1__name, :font__google_fonts__h1__variant, :font__google_fonts__h1__subset, :font__google_fonts__default__name, :font__google_fonts__default__variant, :font__google_fonts__default__subset, :view__offset__height, :view__offset__widht, :style__h1__color__opacity, :style__h1__color__default, :style__h1__color__palette, :style__h1__stroke__color, :style__h1__stroke__width, :style__h1__stroke__opacity, :style__h1__stroke__linecap, :style__default__color__opacity, :style__default__color__default, :style__default__color__palette, :style__default__stroke__color, :style__default__stroke__width, :style__default__stroke__opacity, :style__default__stroke__linecap], [:github__profile, :font__local__h1, :font__local__default, :font__current__h1, :font__current__default]]

## Google Fonts Download

In [ ]:
google_fonts_download( 'Amatic SC', '700', 'latin', 'test.ttf' )

In [15]:
lists[ 0 ]

[:github__source, :font__text_align, :font__bold, :font__mode__h1, :font__mode__default, :font__google_fonts__h1__name, :font__google_fonts__h1__variant, :font__google_fonts__h1__subset, :font__google_fonts__default__name, :font__google_fonts__default__variant, :font__google_fonts__default__subset, :view__offset__height, :view__offset__widht, :style__h1__color__opacity, :style__h1__color__default, :style__h1__color__palette, :style__h1__stroke__color, :style__h1__stroke__width, :style__h1__stroke__opacity, :style__h1__stroke__linecap, :style__default__color__opacity, :style__default__color__default, :style__default__color__palette, :style__default__stroke__color, :style__default__stroke__width, :style__default__stroke__opacity, :style__default__stroke__linecap]

In [77]:
str = "#  # #  test"
str.gsub('#', '')

"     test"

In [21]:
str = 'dedest.svg'
str[ str.length-45, str.length]

In [25]:
s = 1624037044


1624037044

In [ ]:
Time.parse( s )

NoMethodError: undefined method `parse' for Time:Class

In [37]:
s = "default-#{Time.now.getutc.to_i}.ttf"


"default-1624037555.ttf"

In [39]:
Time.now.getutc.to_i - s

true

In [28]:
Time.now.getutc.to_i

1624037260